In [1]:
pip install ogb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:00


In [3]:
pip install torch==2.5.0 --force-reinstall

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
from ogb.linkproppred import LinkPropPredDataset, Evaluator
import numpy as np
from torch_geometric.data import Data
import torch.nn as nn
from torch_geometric.utils import negative_sampling, scatter, add_self_loops, softmax
import torch.nn.functional as F
import torch.nn.init as init
from torch_geometric.nn import MessagePassing
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [5]:
def load_dataset():
    dataset = LinkPropPredDataset(name="ogbl-ddi")
    split_edge = dataset.get_edge_split()
    data = dataset[0]
    return data, split_edge, dataset
data, split_edge, dataset = load_dataset()

Downloaded 0.04 GB: 100%|██████████| 46/46 [00:01<00:00, 41.37it/s]


Extracting dataset/ddi.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 32.61it/s]

Saving...



/usr/local/lib/python3.11/dist-packages/ogb/linkproppred/dataset.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train = torch.load(osp.join(path, 'train.pt'))
/usr/l

In [6]:
edge_index = torch.from_numpy(split_edge['train']['edge']).long().T.contiguous()
num_nodes = 4267
def fast_undirected_negative_sampling(edge_index, num_nodes, num_samples):
    u, v = edge_index
    pos_u = torch.min(u, v)
    pos_v = torch.max(u, v)
    pos_pairs = pos_u * num_nodes + pos_v
    pos_set = pos_pairs.unique()

    neg_set = set()
    max_trials = num_samples * 10
    trials = 0

    while len(neg_set) < num_samples and trials < max_trials:
        i = torch.randint(0, num_nodes, (num_samples * 2,))
        j = torch.randint(0, num_nodes, (num_samples * 2,))
        mask = i != j
        i, j = i[mask], j[mask]

        u = torch.min(i, j)
        v = torch.max(i, j)
        pair_ids = u * num_nodes + v

        valid_mask = ~torch.isin(pair_ids, pos_set)
        u, v, pair_ids = u[valid_mask], v[valid_mask], pair_ids[valid_mask]

        for a, b, pid in zip(u.tolist(), v.tolist(), pair_ids.tolist()):
            if pid not in neg_set:
                neg_set.add(pid)
                if len(neg_set) == num_samples:
                    break
        trials += 1

    neg_pairs = torch.tensor([(pid // num_nodes, pid % num_nodes) for pid in neg_set], dtype=torch.long).t()
    neg_sym = torch.cat([neg_pairs, neg_pairs[[1, 0]]], dim=1)  # add reverse edges
    return neg_sym



In [18]:
num_edges = edge_index.shape[1]

perm = torch.randperm(num_edges)
shuffled_edges = edge_index[:, perm]

split_idx = int(0.8 * num_edges)
message_passing_edges = shuffled_edges[:, :split_idx]
train_supervision_edges = shuffled_edges[:, split_idx:]

def add_reverse_edges(edges):
    return torch.cat([edges, edges[[1, 0]]], dim=1)

message_passing_edges = add_reverse_edges(message_passing_edges)
train_supervision_edges = add_reverse_edges(train_supervision_edges)

print(f"Message Passing Edges Shape: {message_passing_edges.shape}")
print(f"Train Supervision Edges Shape: {train_supervision_edges.shape}")

num_neg_samples = train_supervision_edges.shape[1] // 2
neg_edge_index = fast_undirected_negative_sampling(edge_index, num_nodes, num_neg_samples)
print(f"Negative Edge Index Shape: {neg_edge_index.shape}")


Message Passing Edges Shape: torch.Size([2, 1708656])
Train Supervision Edges Shape: torch.Size([2, 427166])
Negative Edge Index Shape: torch.Size([2, 427166])


In [8]:
class GINLayer(MessagePassing):
    def __init__(self, in_dim, hidden_dim, eps=0):
        super().__init__(aggr="sum")


        self.mlp_phi = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.3),
            nn.ReLU()
        )

        self.eps = nn.Parameter(torch.Tensor([eps]))

    def forward(self, x, edge_index):
        x = x.to(self.mlp_phi[0].weight.device)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        return (x_j)


    def update(self, aggr_out, x):
        out = (1 + self.eps) * (x) + aggr_out
        return self.mlp_phi(out)


class GINLinkPredictor(nn.Module):
    def __init__(self, num_nodes, hidden_dim):
        super().__init__()

        self.node_emb = nn.Embedding(num_nodes, hidden_dim)
        init.xavier_uniform_(self.node_emb.weight)
        self.gin1 = GINLayer(hidden_dim, hidden_dim)
        self.gin2 = GINLayer(hidden_dim, hidden_dim)
        self.gin3 = GINLayer(hidden_dim, hidden_dim)
        self.edge_predictor = nn.Sequential(
                nn.Linear(2 * hidden_dim, 100),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(100, 60),
                nn.ReLU(),
                nn.Linear(60, 30),
                nn.ReLU(),
                nn.Linear(30, 1)
                )


    def forward(self, edge_index):
        x = self.node_emb.weight.to(edge_index.device)

        x = self.gin1(x, edge_index)

        x = self.gin2(x, edge_index)

        x = self.gin3(x, edge_index)

        return x

    def predict_edges(self, x, edge_index):
        src, dst = edge_index
        edge_features = torch.concat([x[src], x[dst]],dim=1)
        return (self.edge_predictor(edge_features).squeeze(-1))

In [9]:
@torch._dynamo.disable
def run_eval(input_dict):
    return evaluator.eval(input_dict)

In [10]:

@torch.compile
def train_step(model, optimizer, scheduler, edge_index, train_edges, neg_edges, val_edges, val_neg_edges, device):
    model.train()
    optimizer.zero_grad()

    x = model(edge_index.to(device))

    pos = model.predict_edges(x, train_edges)
    neg = model.predict_edges(x, neg_edges)

    scores = torch.cat([pos, neg], dim=0)
    labels = torch.cat([
        torch.ones_like(pos),
        torch.zeros_like(neg)
    ], dim=0)

    bce_loss_fn = nn.BCEWithLogitsLoss()
    loss = bce_loss_fn(scores, labels)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2)

    optimizer.step()
    scheduler.step()
    model.eval()
    with torch.no_grad():
        x = model(edge_index.to(device))
        pos_val = model.predict_edges(x, val_edges.to(device))
        neg_val = model.predict_edges(x, val_neg_edges.to(device))
        input_dict = {"y_pred_pos": pos_val.view(-1), "y_pred_neg": neg_val.view(-1)}
        result = run_eval(input_dict)
    return loss, result["hits@20"]

In [11]:
@torch.no_grad()
@torch.compile
def test_step(model, edge_index, test_pos_edges, test_neg_edges, evaluator, device):
    model.eval()
    x = model(edge_index.to(device))

    pos_preds = model.predict_edges(x, test_pos_edges.to(device))
    neg_preds = model.predict_edges(x, test_neg_edges.to(device))

    avg_pos_score = pos_preds.mean().item()
    avg_neg_score = neg_preds.mean().item()
    top20_neg_avg = torch.topk(neg_preds, 20, largest=True).values.mean().item()

    print(f"🔹 Avg Positive Edge Score: {avg_pos_score:.15f}")
    print(f"🔻 Avg Negative Edge Score: {avg_neg_score:.15f}")
    print(f"🔺 Avg Top 20 Negative Edge Score: {top20_neg_avg:.15f}")

    input_dict = {"y_pred_pos": pos_preds.view(-1), "y_pred_neg": neg_preds.view(-1)}
    result = evaluator.eval(input_dict)

    pos_labels = torch.ones_like(pos_preds)
    neg_labels = torch.zeros_like(neg_preds)

    y_true = torch.cat([pos_labels, neg_labels], dim=0)
    y_pred = torch.cat([pos_preds, neg_preds], dim=0)
    y_pred = (torch.sigmoid(y_pred) > 0.5).float()

    correct = (y_pred == y_true).sum().item()
    accuracy = correct / y_true.numel()

    true_positives = ((y_pred == 1) & (y_true == 1)).sum().item()
    predicted_positives = (y_pred == 1).sum().item()
    actual_positives = (y_true == 1).sum().item()

    precision = true_positives / predicted_positives if predicted_positives > 0 else 0.0
    recall = true_positives / actual_positives if actual_positives > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"🎯 Precision: {precision:.4f}")
    print(f"🔄 Recall: {recall:.4f}")
    print(f"⭐ F1 Score: {f1:.4f}")

    return result["hits@20"]


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
val_edges = torch.tensor(split_edge['test']['edge'].T, dtype=torch.long, device=device)
val_neg_edges = torch.tensor(split_edge['test']['edge_neg'].T, dtype=torch.long, device=device)

In [13]:
SAVE_PATH = "path_to_model"

In [ ]:
import os

model = GINLinkPredictor(num_nodes=4267, hidden_dim=256).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
model.compile()
print(f"Using device: {device}")
best = 0.0
num_epochs=1500

for epoch in range(num_epochs):
    loss, val_hits_at_20 = train_step(model, optimizer, scheduler, message_passing_edges, train_supervision_edges, neg_edge_index,val_edges, val_neg_edges, device)
    print(f"Epoch {epoch+1}, Train Loss: {loss:.15f}, Val hits at 20: {val_hits_at_20:.15f}")

    if val_hits_at_20 > best:
        best = val_hits_at_20
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"Model saved at epoch {epoch+1} with loss {loss:.15f}")


In [14]:
def load_model(save_path, num_nodes, device):
    model = GINLinkPredictor(num_nodes=num_nodes, hidden_dim=256).to(device)
    model.load_state_dict(torch.load(save_path, map_location=device, weights_only=False))
    model.eval()

    return model

In [15]:
evaluator = Evaluator(name="ogbl-ddi")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
loaded_model = load_model(SAVE_PATH, num_nodes=4267, device=device)


GPU available: True
GPU name: Tesla T4


In [22]:
message_passing_edges = torch.load("path_to_message_passing_edges",weights_only = False).to(device)


In [23]:
test_pos_edges = torch.tensor(split_edge['test']['edge'].T, dtype=torch.long, device=device)
test_neg_edges = torch.tensor(split_edge['test']['edge_neg'].T, dtype=torch.long, device=device)

hits_at_20 = test_step(loaded_model, message_passing_edges, test_pos_edges, test_neg_edges, evaluator, device)
print(f"Test Hits@20 (OGB Evaluator): {hits_at_20:.15f}")

🔹 Avg Positive Edge Score: 2.397875785827637
🔻 Avg Negative Edge Score: -5.865193367004395
🔺 Avg Top 20 Negative Edge Score: 3.070761203765869
✅ Accuracy: 0.9566
🎯 Precision: 0.9657
🔄 Recall: 0.9596
⭐ F1 Score: 0.9626
Test Hits@20 (OGB Evaluator): 0.313269258141120


In [24]:
val_edges = torch.tensor(split_edge['valid']['edge'].T, dtype=torch.long, device=device)
val_neg_edges = torch.tensor(split_edge['valid']['edge_neg'].T, dtype=torch.long, device=device)

hits_at_20 = test_step(loaded_model, message_passing_edges, val_edges, val_neg_edges, evaluator, device)
print(f"Test Hits@20 (OGB Evaluator): {hits_at_20:.15f}")

🔹 Avg Positive Edge Score: 3.088223695755005
🔻 Avg Negative Edge Score: -5.980741977691650
🔺 Avg Top 20 Negative Edge Score: 3.908168554306030
✅ Accuracy: 0.9580
🎯 Precision: 0.9556
🔄 Recall: 0.9711
⭐ F1 Score: 0.9633
Test Hits@20 (OGB Evaluator): 0.356036827004472


In [17]:
def print_trainable_params(model):
    print("Trainable Parameters:")
    total_params = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"{name}: {param.shape} | {param.numel()} params")
            total_params += param.numel()

    print(f"Total Trainable Parameters: {total_params}")

print_trainable_params(loaded_model)


Trainable Parameters:
node_emb.weight: torch.Size([4267, 256]) | 1092352 params
gin1.eps: torch.Size([1]) | 1 params
gin1.mlp_phi.0.weight: torch.Size([256, 256]) | 65536 params
gin1.mlp_phi.0.bias: torch.Size([256]) | 256 params
gin1.mlp_phi.1.weight: torch.Size([256]) | 256 params
gin1.mlp_phi.1.bias: torch.Size([256]) | 256 params
gin1.mlp_phi.4.weight: torch.Size([256, 256]) | 65536 params
gin1.mlp_phi.4.bias: torch.Size([256]) | 256 params
gin1.mlp_phi.5.weight: torch.Size([256]) | 256 params
gin1.mlp_phi.5.bias: torch.Size([256]) | 256 params
gin2.eps: torch.Size([1]) | 1 params
gin2.mlp_phi.0.weight: torch.Size([256, 256]) | 65536 params
gin2.mlp_phi.0.bias: torch.Size([256]) | 256 params
gin2.mlp_phi.1.weight: torch.Size([256]) | 256 params
gin2.mlp_phi.1.bias: torch.Size([256]) | 256 params
gin2.mlp_phi.4.weight: torch.Size([256, 256]) | 65536 params
gin2.mlp_phi.4.bias: torch.Size([256]) | 256 params
gin2.mlp_phi.5.weight: torch.Size([256]) | 256 params
gin2.mlp_phi.5.bias: t